In [592]:
import pandas as pd 
import numpy as np
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import cross_val_score,GridSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score,f1_score

In [593]:
df_train=pd.read_csv('data/train.csv')
df_test=pd.read_csv('data/test.csv')

In [594]:
df_test

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [595]:
df_train.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

# show distribution of values in each columns

In [596]:
for column in df_train.columns:
    display(df_train[column].value_counts())
    display(df_train[column].value_counts())
    print()

0001_01    1
6136_01    1
6141_01    1
6139_06    1
6139_05    1
          ..
3126_01    1
3124_03    1
3124_02    1
3124_01    1
9280_02    1
Name: PassengerId, Length: 8693, dtype: int64

0001_01    1
6136_01    1
6141_01    1
6139_06    1
6139_05    1
          ..
3126_01    1
3124_03    1
3124_02    1
3124_01    1
9280_02    1
Name: PassengerId, Length: 8693, dtype: int64

Earth     4602
Europa    2131
Mars      1759
Name: HomePlanet, dtype: int64

Earth     4602
Europa    2131
Mars      1759
Name: HomePlanet, dtype: int64

False    5439
True     3037
Name: CryoSleep, dtype: int64

...

0.0       5495
1.0        139
2.0         70
3.0         56
5.0         51
          ... 
408.0        1
876.0        1
2891.0       1
2102.0       1
3235.0       1
Name: VRDeck, Length: 1306, dtype: int64

Gollux Reedall        2
Elaney Webstephrey    2
Grake Porki           2
Sus Coolez            2
Apix Wala             2
                     ..
Jamela Griffy         1
Hardy Griffy          1
Salley Mckinn         1
Mall Frasp            1
Propsh Hontichre      1
Name: Name, Length: 8473, dtype: int64

Gollux Reedall        2
Elaney Webstephrey    2
Grake Porki           2
Sus Coolez            2
Apix Wala             2
                     ..
Jamela Griffy         1
Hardy Griffy          1
Salley Mckinn         1
Mall Frasp            1
Propsh Hontichre      1
Name: Name, Length: 8473, dtype: int64

True     4378
False    4315
Name: Transported, dtype: int64

True     4378
False    4315
Name: Transported, dtype: int64

### as we see above , some of the columns distributed bad, but we can see here that most of them are numeric and most used one is 0 ,  so we can fill these nulls with 0 

In [597]:
df_train['VIP']=df_train['VIP'].fillna(False)
df_test['VIP']=df_test['VIP'].fillna(False)

In [598]:
df_train['RoomService']=df_train['RoomService'].fillna(0.0)
df_test['RoomService']=df_test['RoomService'].fillna(0.0)

In [599]:
df_train['FoodCourt']=df_train['FoodCourt'].fillna(0.0)
df_test['FoodCourt']=df_test['FoodCourt'].fillna(0.0)

In [600]:
df_train['ShoppingMall']=df_train['ShoppingMall'].fillna(0.0)
df_test['ShoppingMall']=df_test['ShoppingMall'].fillna(0.0)

In [601]:
df_train['Spa']=df_train['Spa'].fillna(0.0)
df_test['Spa']=df_test['Spa'].fillna(0.0)

In [602]:
df_train['VRDeck']=df_train['VRDeck'].fillna(0.0)
df_test['VRDeck']=df_test['VRDeck'].fillna(0.0)

In [603]:
df_train.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            200
Transported       0
dtype: int64

In [604]:
len(df_train.dropna())/len(df_train)

0.8715058092718279

In [605]:
len(df_test.dropna())/len(df_test)

0.8765489829319617

In [606]:
df_train.HomePlanet.value_counts()

Earth     4602
Europa    2131
Mars      1759
Name: HomePlanet, dtype: int64

In [607]:
df_train=df_train.dropna()
#df_test=df_test.dropna()

In [608]:
df_test['HomePlanet']=df_test['HomePlanet'].fillna('Earth')
df_test['CryoSleep']=df_test['CryoSleep'].fillna(False)
df_test['Cabin']=df_test['Cabin'].fillna('G/160/P')
df_test['Destination']=df_test['Destination'].fillna('TRAPPIST-1e')
df_test['Age']=df_test['Age'].fillna(25.8)   # mean 28.5 # top20 mean 25.8

In [610]:
#df_test.RoomService.value_counts()

In [611]:
#df_train.Age.value_counts().to_frame().head(20)

In [612]:
df_train['Cabin_1']=0
df_train['Cabin_2']=0
df_train['Cabin_3']=0
for index,row in df_train.iterrows():
    tmp_cabin_list=row['Cabin'].split('/')
    df_train.at[index,'Cabin_1']=tmp_cabin_list[0]
    df_train.at[index,'Cabin_2']=tmp_cabin_list[1]
    df_train.at[index,'Cabin_3']=tmp_cabin_list[2]

In [613]:
df_test['Cabin_1']=0
df_test['Cabin_2']=0
df_test['Cabin_3']=0
for index,row in df_test.iterrows():
    tmp_cabin_list=row['Cabin'].split('/')
    df_test.at[index,'Cabin_1']=tmp_cabin_list[0]
    df_test.at[index,'Cabin_2']=tmp_cabin_list[1]
    df_test.at[index,'Cabin_3']=tmp_cabin_list[2]

In [614]:
df_train.drop(columns=['Cabin'],inplace=True)
df_test.drop(columns=['Cabin'],inplace=True)

In [615]:
df_train['Cabin_2']=df_train.Cabin_2.apply(pd.to_numeric)
df_test['Cabin_2']=df_test.Cabin_2.apply(pd.to_numeric)

# Work with Name columns

In [616]:
all_name=list(set(df_train.Name).union(set(df_test.Name)))

In [617]:
names_with_id=dict()
for i in range(len(all_name)):
    names_with_id[all_name[i]]=i

In [618]:
df_train['Name_encoded']=0
for index,row in df_train.iterrows():
    df_train.at[index,'Name_encoded']=names_with_id[row['Name']]

In [619]:
df_test['Name_encoded']=0
for index,row in df_test.iterrows():
    df_test.at[index,'Name_encoded']=names_with_id[row['Name']]

In [620]:
df_train.drop(columns=['Name'],inplace=True)
df_test.drop(columns=['Name'],inplace=True)

In [621]:
df_train.drop(columns=['Name_encoded'],inplace=True)
df_test.drop(columns=['Name_encoded'],inplace=True)

In [622]:
# Multiple categorical columns
categorical_cols = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP','Cabin_1','Cabin_3']

df_train=pd.get_dummies(df_train, columns=categorical_cols).copy()
df_test=pd.get_dummies(df_test, columns=categorical_cols).copy()

In [623]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

### For local test train the train data 

In [624]:
# splitting X and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_train.drop(columns=['PassengerId','Transported']),
                                                    df_train['Transported'], test_size=0.3, random_state=42)


In [625]:
# training the model on training set
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# making predictions on the testing set
y_pred = gnb.predict(X_test)
  
# comparing actual response values (y_test) with predicted response values (y_pred)
print("Gaussian Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_test, y_pred)*100)

Gaussian Naive Bayes model accuracy(in %): 75.14298284205894


# hyperparameter tuning

In [626]:
# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [627]:
space={'max_depth': hp.quniform("max_depth", 3, 10, 1),
        'gamma': hp.uniform ('gamma', 1,20),
        'learning_rate': hp.quniform('learning_rate', 0,2,0.01),
        'max_delta_step':hp.uniform ('max_delta_step', 0,10),
        'reg_alpha' : hp.quniform('reg_alpha', 0,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'min_child_weight' : hp.uniform('min_child_weight', 0, 10),
        'n_estimators': 100,
        'seed': 10
    }    

In [628]:
def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'],
                    max_depth = int(space['max_depth']), 
                    gamma = space['gamma'],
                    reg_alpha = space['reg_alpha'],
                    min_child_weight=int(space['min_child_weight']),
                    max_delta_step=int(space['max_delta_step']),
                    learning_rate=space['learning_rate'],
                    seed=space['seed'],
                    eval_metric='auc',
                    early_stopping_rounds=15
    )
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation,
            verbose=False)
    

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [629]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

SCORE:                                                                          
0.7699076110866696                                                              
SCORE:                                                                          
0.7932248130224373                                                              
SCORE:                                                                          
0.7773867135943687                                                              
SCORE:                                                                          
0.796304443466784                                                               
SCORE:                                                                          
0.7971843378794544                                                              
SCORE:                                                                          
0.7936647602287725                                                              
SCORE:                      

0.8011438627364716                                                              
SCORE:                                                                          
0.7809062912450506                                                              
SCORE:                                                                          
0.7699076110866696                                                              
SCORE:                                                                          
0.7993840739111306                                                              
SCORE:                                                                          
0.7857457105147382                                                              
SCORE:                                                                          
0.7927848658161021                                                              
SCORE:                                                                          
0.8068631764188298          

In [630]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

The best hyperparameters are :  

{'gamma': 6.292394181673046, 'learning_rate': 0.72, 'max_delta_step': 4.620635033415536, 'max_depth': 9.0, 'min_child_weight': 3.6258537974193343, 'reg_alpha': 4.0, 'reg_lambda': 0.5948154199928402}


In [631]:
clf=xgb.XGBClassifier(
    max_depth = int(best_hyperparams['max_depth']),
    gamma = best_hyperparams['gamma'],
    reg_alpha = best_hyperparams['reg_alpha'],
    min_child_weight=best_hyperparams['min_child_weight'],
    learning_rate=best_hyperparams['learning_rate'],
    max_delta_step=best_hyperparams['max_delta_step'],
    
    
)
evaluation = [( X_train, y_train), ( X_test, y_test)]
    
clf.fit(X_train, y_train,
        eval_set=evaluation,
        verbose=False)
    

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred>0.5)
print ("SCORE:", accuracy)

SCORE: 0.807303123625165


In [632]:
print(confusion_matrix(y_test, y_pred), ": is the confusion matrix")
print(accuracy_score(y_test, y_pred), ": is the accuracy score")
print(precision_score(y_test, y_pred), ": is the precision score")
print(recall_score(y_test, y_pred), ": is the recall score")
print(f1_score(y_test, y_pred), ": is the f1 score")

[[915 233]
 [205 920]] : is the confusion matrix
0.807303123625165 : is the accuracy score
0.797918473547268 : is the precision score
0.8177777777777778 : is the recall score
0.8077260755048288 : is the f1 score


# For submittion

In [467]:
y_pred_sub = clf.predict(df_test.drop(columns=['PassengerId']))

In [468]:
y_pred_sub

array([1, 0, 1, ..., 1, 1, 1])

In [469]:
y = 0 < y_pred_sub
y

array([ True, False,  True, ...,  True,  True,  True])

# Save result

In [448]:
df_submit=df_test[['PassengerId']].copy()
df_submit['Transported']=y

In [470]:
df_submit.to_csv('data/submit_3.csv',index = False)

In [471]:
df_submit

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True
